In [1]:
import os
import numpy as np
import pandas as pd
from keras.models import load_model
from skimage.transform import resize
from skimage.io import imread
from PIL import Image

C:\Users\Santos\Anaconda3\envs\dl\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Instructions for updating:
Use the retry module or similar alternatives.


In [57]:
def data_preprocess(img_dir, csv_dir, file_prefix="affectnet_landmarks", max_samples=1000):
    print("Start preprocess")
    filenames = [entry.name for entry in os.scandir(csv_dir) if
                 entry.name.lower().endswith(".csv") and entry.name.startswith(file_prefix)]
    x = []
    y = []
    for f in filenames:
        print("Reading file {0}".format(f))
        data_df = pd.read_csv(os.path.join(csv_dir, f))
        for i in range(5000, min(data_df.shape[0], 5000 + max_samples)):
            f = os.path.join(img_dir, str(data_df.loc[i, "subDirectory"]), str(data_df.loc[i, "filePath"]))
            if not os.path.isfile(f):
                continue
            img = imread(f, as_grey=True)
            img = resize(img, (32, 32))
            x.append(img)
            y.append(data_df.loc[i, "expression"])
    # x = np.asarray(x, dtype=np.float32) / 255.0
    x = np.expand_dims(x, axis=-1)
    y = np.asarray(y, dtype=np.int32)
    print("Saving data")
    np.save(os.path.join("x_test.npy"), x)
    np.save(os.path.join("y_test.npy"), y)
    print("Done preprocess")

In [58]:
model = load_model("model.h5")
model.load_weights("weights.h5")

In [59]:
classes = [1, 2, 3, 6]
classes = np.asarray(classes)
num_classes = max(classes) + 1

In [60]:
# data_preprocess(r"E:\final_project_ee\Manually Annotated\Manually_Annotated_Images",
#                 r"C:\Users\Santos\Documents\GitHub\ExpressionRecognition\Affectnet")
x = np.load("x_test.npy")
y = np.load("y_test.npy")

In [61]:
indices = np.where(sum(y == c for c in classes))
x = x[indices]
y = y[indices]

In [62]:
res = model.predict(x)

In [63]:
y_res = np.asarray([classes[np.argmax(v)] for v in res])
sum(y_res == y) / len(y)

0.673

In [64]:
res_mat = [[0 for i in range(num_classes)] for j in range(num_classes)]
for i in range(len(y)):
    res_mat[y[i]][y_res[i]] += 1
res_mat = np.asarray(res_mat)
res_mat

array([[  0,   0,   0,   0,   0,   0,   0],
       [  0, 828,  54,  72,   0,   0,  46],
       [  0,  55, 554, 161,   0,   0, 230],
       [  0,  55, 146, 671,   0,   0, 128],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,  55, 181, 125,   0,   0, 639]])

In [65]:
y2_res = np.asarray([classes[np.argsort(v)[-2:]] for v in res])
sum(y[i] in y2_res[i] for i in range(len(y))) / len(y)

0.885

In [66]:
l = [[0 for i in range(num_classes)] for j in range(num_classes)]
for pair in y2_res:
    l[pair[0]][pair[1]] += 1
    l[pair[1]][pair[0]] += 1
l = np.asarray(l)
l

array([[   0,    0,    0,    0,    0,    0,    0],
       [   0,    0,  331,  747,    0,    0,  274],
       [   0,  331,    0,  946,    0,    0, 1186],
       [   0,  747,  946,    0,    0,    0,  516],
       [   0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0],
       [   0,  274, 1186,  516,    0,    0,    0]])

In [67]:
mis_indices = np.where(y != y_res)
mis_x = x[mis_indices]
mis_y = y[mis_indices]
mis_y_res = y_res[mis_indices]
mis_y2_res = y2_res[mis_indices]

In [68]:
l = [[0 for i in range(num_classes)] for j in range(num_classes)]
for pair in mis_y2_res:
    l[pair[0]][pair[1]] += 1
    l[pair[1]][pair[0]] += 1
l = np.asarray(l)
l

array([[  0,   0,   0,   0,   0,   0,   0],
       [  0,   0, 110, 124,   0,   0,  76],
       [  0, 110,   0, 341,   0,   0, 442],
       [  0, 124, 341,   0,   0,   0, 215],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,  76, 442, 215,   0,   0,   0]])

In [69]:
# data = (mis_x[2].reshape((32, 32))*255).astype(np.int32)
# img = Image.fromarray(data, 'I')
# img.show()